In [180]:
from keys import client_id, client_secret, user_name
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import oauth2
import random
from functools import reduce

In [181]:
scope = 'user-library-read playlist-modify-public playlist-modify-private playlist-read-private user-read-recently-played app-remote-control user-top-read'

redirect_uri = 'https://developer.spotify.com/dashboard/applications/0743a195f7654b5ab95560a95e89316a'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

token = util.prompt_for_user_token(user_name, scope, client_id, client_secret, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("NO TOKEN FOUND")


In [182]:
user_top_tracks = sp.current_user_top_tracks(limit=50, offset=0, time_range='long_term')
song_data = user_top_tracks["items"]

song_ids = []
song_names = []

for i in range(0, len(song_data)):
    if song_data[i]['id'] != None:
        song_ids.append(song_data[i]['id'])
        song_names.append(song_data[i]['name'])

song_features = []
for i in range(0, len(song_ids)):
    features = sp.audio_features(song_ids[i])
    for song in features:
        song_features.append(song)

top_tracks_df = pd.DataFrame(song_features, index=song_names)
top_tracks_df = top_tracks_df[["id", "acousticness", "danceability", "duration_ms", 
                         "energy", "instrumentalness",  "key", "liveness",
                         "loudness", "mode", "speechiness", "tempo", "valence"]]
top_tracks_df.head()

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
HOLIDAY,6zFMeegAMYQo0mt8rXtrli,0.12000,0.810,154998,0.511,0.000000,5,0.0832,-6.924,0,0.1640,151.947,0.837
Death of a Bachelor,1BECwm5qkaBwlbfo4kpYx8,0.01370,0.462,203507,0.538,0.000000,0,0.4290,-5.527,1,0.0590,139.256,0.405
Daylight,6Ed1q0X8oSKSm4IIhiQbYg,0.00488,0.528,163906,0.749,0.000096,7,0.0949,-7.571,1,0.0479,163.944,0.729
Knock Knock,3uYm4MtU6jUQft2DtGqEoZ,0.03530,0.667,202807,0.922,0.000000,4,0.3640,-4.857,0,0.1280,108.952,0.509
Don't Matter,7I6DceMT3utDOHjcYCbrr4,0.24400,0.798,293053,0.443,0.000000,2,0.3260,-6.062,1,0.0403,125.232,0.354


In [183]:
top_tracks_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, HOLIDAY to Team
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                50 non-null     object 
 1   acousticness      50 non-null     float64
 2   danceability      50 non-null     float64
 3   duration_ms       50 non-null     int64  
 4   energy            50 non-null     float64
 5   instrumentalness  50 non-null     float64
 6   key               50 non-null     int64  
 7   liveness          50 non-null     float64
 8   loudness          50 non-null     float64
 9   mode              50 non-null     int64  
 10  speechiness       50 non-null     float64
 11  tempo             50 non-null     float64
 12  valence           50 non-null     float64
dtypes: float64(9), int64(3), object(1)
memory usage: 5.5+ KB


In [184]:
featured_playlists = sp.featured_playlists(limit=50)
# Creating data frame for playlists
id = []
name = []
num_tracks = []
items = featured_playlists['playlists']['items']
for item in items:
    id.append(item["id"])
    name.append(item["name"])
    num_tracks.append(item["tracks"]["total"])

df_pl_data = pd.DataFrame({"id":id, "name": name, "num_tracks": num_tracks})

In [185]:
df_pl_data

,id,name,num_tracks
0,37i9dQZF1DWSRc3WJklgBs,Heart Beats,198
1,37i9dQZF1DX843Qf4lrFtZ,"Young, Wild & Free",100
2,37i9dQZF1DX4bSrsRWE9cd,Bliss,75
3,37i9dQZF1DX1BGjF5N5O7F,Sexy As Folk,50
4,37i9dQZF1DWZd79rJ6a7lp,Sleep,186
5,37i9dQZF1DX5Ejj0EkURtP,All Out 10s,150
6,37i9dQZF1DX4WYpdgoIcn6,Chill Hits,130
7,37i9dQZF1DWY4xHQp97fN6,Get Turnt,100
8,37i9dQZF1DX2yvmlOdMYzV,Low-Key,100
9,37i9dQZF1DX1aA8os1ylFZ,BAE,100


In [186]:
# Returns Individual Tracks In Playlists
def get_playlist_tracks(sp, playlist_id):
    data_track = sp.playlist_tracks(playlist_id, fields=None, limit=100, offset=0, market=None)['items']
    track_id = []
    track_name = []
    
    for data in data_track:
        track_id.append(data['track']['id'])
        track_name.append(data['track']['name'])

    return pd.DataFrame({"track_id":track_id, "track_name": track_name})

In [187]:
get_playlist_tracks(sp, '37i9dQZF1DWZKuerrwoAGz').head()

,track_id,track_name
0,6FE2iI43OZnszFLuLtvvmg,Classic
1,3DmW6y7wTEYHJZlLo1r6XJ,Shower
2,4E5P1XyAFtrjpiIxkydly4,Replay
3,3E7dfMvvCLUddWissuqMwr,Party In The U.S.A.
4,1CQ2cMfrmFM1YdfmjENKVE,She Looks So Perfect


In [188]:
# Function for getting audio features of individual tracks
def get_audio_features(sp, playlist_id):
    playlist = get_playlist_tracks(sp, playlist_id)
    audio_features = []
    for i in range(len(playlist)):
        song_data_id = playlist['track_id'][i]
        audio_features.append(sp.audio_features(song_data_id))
    features_array = []
    for features in audio_features:
        features = features[0]
        features_array.append([features['danceability'],
                              features['acousticness'],
                              features['energy'], 
                              features['tempo'],
                              features['instrumentalness'], 
                              features['loudness'],
                              features['liveness'],
                              features['duration_ms'],
                              features['key'],
                              features['valence'],
                              features['speechiness']
                             ])
    df_audio_features = pd.DataFrame(features_array, columns=['danceability', 'acousticness', 'energy', 'tempo', 'instrumentalness', 'loudness', 'liveness', 'duration_ms', 'key', 'valence', 'speechiness'])
    # DataFrame with audio features and playlist id/name
    df_playlist_features = pd.concat([playlist, df_audio_features], axis=1)
    df_playlist_features.set_index('track_name', inplace=True, drop=True)
    return df_playlist_features

In [189]:
get_audio_features(sp, '37i9dQZF1DXcRXFNfZr7Tp').head()

,track_id,danceability,acousticness,energy,tempo,instrumentalness,loudness,liveness,duration_ms,key,valence,speechiness
track_name,,,,,,,,,,,,
Beautiful Mistakes (feat. Megan Thee Stallion),6fRxMU4LWwyaSSowV441IU,0.713,0.0377,0.676,99.048,0.000000,-5.483,0.154,227395,10,0.721,0.0270
Met Him Last Night (feat. Ariana Grande),0BI0hfbmqybnd3TezrDME3,0.538,0.2200,0.512,144.978,0.000000,-4.548,0.101,204632,4,0.120,0.0262
BED,7jXQUrVhEpXdymfFWNDnQW,0.663,0.0134,0.783,123.986,0.001790,-4.585,0.325,178088,6,0.622,0.0393
Peaches (feat. Daniel Caesar & Giveon),4iJyoBOLtHqaGxP12qzhQI,0.677,0.3210,0.696,90.030,0.000000,-6.181,0.420,198082,0,0.464,0.1190
deja vu,61KpQadow081I2AsbeLcsb,0.439,0.5930,0.610,181.088,0.000011,-7.236,0.341,215508,9,0.172,0.1160


In [190]:
# Finding mean of audio features for each playlist
def audio_features_mean(sp, playlist_id):
    Playlist = get_audio_features(sp, playlist_id)
    return pd.DataFrame(Playlist.mean(), columns= [playlist_id])

In [191]:
audio_features_mean(sp, '37i9dQZF1DXbYM3nMM0oPk')

,37i9dQZF1DXbYM3nMM0oPk
danceability,0.709173
acousticness,0.181992
energy,0.645507
tempo,121.424267
instrumentalness,0.008807
loudness,-5.947773
liveness,0.162463
duration_ms,188870.400000
key,5.253333
valence,0.545632


In [192]:
# Merges average of audio features of each playlist into a single DataFrame
primary_df = []
for i in range(len(df_pl_data)):
    primary_df.append(audio_features_mean(sp, df_pl_data['id'][i]))

In [193]:
primary_df

[                  37i9dQZF1DWSRc3WJklgBs
 danceability                    0.617610
 acousticness                    0.107817
 energy                          0.766540
 tempo                         123.798780
 instrumentalness                0.025127
 loudness                       -5.508830
 liveness                        0.180683
 duration_ms                188617.020000
 key                             5.890000
 valence                         0.414486
 speechiness                     0.067123,
                   37i9dQZF1DX843Qf4lrFtZ
 danceability                    0.677310
 acousticness                    0.160722
 energy                          0.691900
 tempo                         119.616990
 instrumentalness                0.000160
 loudness                       -5.372790
 liveness                        0.170980
 duration_ms                178213.750000
 key                             5.480000
 valence                         0.562920
 speechiness                     

In [194]:
# Turns playlist feature mean data into correctly formatted DataFrame
X_data = reduce(lambda left,right: pd.merge(left,right, left_index=True, right_index=True), primary_df)
X_data

,37i9dQZF1DWSRc3WJklgBs,37i9dQZF1DX843Qf4lrFtZ,37i9dQZF1DX4bSrsRWE9cd,37i9dQZF1DX1BGjF5N5O7F,37i9dQZF1DWZd79rJ6a7lp,37i9dQZF1DX5Ejj0EkURtP,37i9dQZF1DX4WYpdgoIcn6,37i9dQZF1DWY4xHQp97fN6,37i9dQZF1DX2yvmlOdMYzV,37i9dQZF1DX1aA8os1ylFZ,37i9dQZF1DX0QKpU3cGsyb,37i9dQZF1DX6xOPeSOGone
danceability,0.617610,0.677310,0.493147,0.527700,0.160678,0.684540,0.650250,0.806070,0.674990,0.583160,0.624714,0.529590
acousticness,0.107817,0.160722,0.816849,0.604032,0.944250,0.137194,0.356874,0.115721,0.419566,0.508304,0.330599,0.279832
energy,0.766540,0.691900,0.235191,0.397290,0.032744,0.728110,0.525280,0.621680,0.479430,0.454160,0.515786,0.557610
tempo,123.798780,119.616990,118.281827,119.284680,81.433500,117.070070,116.588920,128.961100,108.973900,114.335140,117.082914,120.550960
instrumentalness,0.025127,0.000160,0.162121,0.040691,0.883610,0.002972,0.005211,0.006484,0.041125,0.002175,0.003760,0.013119
loudness,-5.508830,-5.372790,-14.750040,-10.190840,-31.683130,-5.043100,-7.176000,-6.459370,-8.804380,-8.342350,-8.000343,-8.364110
liveness,0.180683,0.170980,0.132576,0.158820,0.106823,0.178658,0.172499,0.163975,0.149906,0.161194,0.164307,0.154143
duration_ms,188617.020000,178213.750000,231474.000000,226657.980000,185802.540000,214663.690000,186238.160000,182556.170000,198020.250000,193878.200000,204737.700000,251054.760000
key,5.890000,5.480000,3.813333,4.420000,5.060000,5.230000,4.990000,4.640000,5.350000,4.980000,5.285714,4.870000
valence,0.414486,0.562920,0.249524,0.330820,0.077998,0.590890,0.444416,0.497988,0.462918,0.370918,0.424427,0.431618


In [195]:
# Turns my favorite track data (Y_data) into a DataFrame with audio features averaged
# maybe change mean to median
instrumental_mean = top_tracks_df['instrumentalness'].mean()
Y_data = pd.DataFrame(top_tracks_df.median(), columns=['top_tracks']).drop('mode')
Y_data.loc['instrumentalness'] = instrumental_mean
Y_data

,top_tracks
acousticness,0.038250
danceability,0.655500
duration_ms,204280.000000
energy,0.701500
instrumentalness,0.000695
key,7.000000
liveness,0.127500
loudness,-5.165500
speechiness,0.060100
tempo,120.641500


In [196]:
# Model for predicting playlist of "best-fit" using feature_importances_
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=44, max_depth=5, max_features=8, oob_score=True)
model.fit(X_data, Y_data['top_tracks'])
pl_rank = model.feature_importances_
pl_importances = pd.DataFrame(pl_rank, index = X_data.columns, columns=['importance']).sort_values('importance',                                        ascending=False)

pl_importances

,importance
37i9dQZF1DWZd79rJ6a7lp,0.353991
37i9dQZF1DX4bSrsRWE9cd,0.108516
37i9dQZF1DX6xOPeSOGone,0.083970
37i9dQZF1DX1BGjF5N5O7F,0.082857
37i9dQZF1DX5Ejj0EkURtP,0.059033
37i9dQZF1DWSRc3WJklgBs,0.058453
37i9dQZF1DX4WYpdgoIcn6,0.058306
37i9dQZF1DX843Qf4lrFtZ,0.055326
37i9dQZF1DWY4xHQp97fN6,0.050435
37i9dQZF1DX0QKpU3cGsyb,0.036293


In [197]:
pl_t3 = pl_importances.index[0:3]
frames = []
for i in range(len(pl_t3)):
    frame = get_audio_features(sp, pl_t3[i])
    frames.append(frame)
    
recommended_pl = pd.concat(frames)

In [198]:
recommended_pl.head()

,track_id,danceability,acousticness,energy,tempo,instrumentalness,loudness,liveness,duration_ms,key,valence,speechiness
track_name,,,,,,,,,,,,
Viento Estelar,0f5IlhiVWUUq4SGHyrRxIt,0.141,0.965,0.04050,59.418,0.966,-29.013,0.0983,185943,3,0.0378,0.0382
Beautiful Imagination,5LEHlRPjGcZ5RdagAwXpHS,0.102,0.797,0.03330,89.605,0.927,-34.177,0.1110,143053,3,0.0591,0.0476
Nepenthe,0kKOC50fPTnqmrYvtJInsT,0.133,0.993,0.00512,67.818,0.851,-33.295,0.0954,151750,9,0.0495,0.0380
Lenitivo,6PEwEkL4lzRbGMGH7j7kBB,0.183,0.987,0.01230,72.183,0.785,-34.571,0.0869,180625,9,0.0636,0.0409
Campus,2PUoynqedBx15JbPYxzA5v,0.145,0.878,0.01150,72.120,0.855,-28.292,0.1220,219474,0,0.0320,0.0421


In [199]:
pl_rec = recommended_pl.set_index('track_id')
pl_rec.sort_index( axis=1, level=None, ascending=True, inplace=True, kind='quicksort')

pl_rec.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence
track_id,,,,,,,,,,,
0f5IlhiVWUUq4SGHyrRxIt,0.965,0.141,185943,0.04050,0.966,3,0.0983,-29.013,0.0382,59.418,0.0378
5LEHlRPjGcZ5RdagAwXpHS,0.797,0.102,143053,0.03330,0.927,3,0.1110,-34.177,0.0476,89.605,0.0591
0kKOC50fPTnqmrYvtJInsT,0.993,0.133,151750,0.00512,0.851,9,0.0954,-33.295,0.0380,67.818,0.0495
6PEwEkL4lzRbGMGH7j7kBB,0.987,0.183,180625,0.01230,0.785,9,0.0869,-34.571,0.0409,72.183,0.0636
2PUoynqedBx15JbPYxzA5v,0.878,0.145,219474,0.01150,0.855,0,0.1220,-28.292,0.0421,72.120,0.0320


In [200]:
Y_data_sorted = Y_data.sort_index( axis=0, level=None, ascending=True, inplace=False, kind='quicksort').squeeze(1)

Y_data_sorted.shape

(11,)

In [201]:
Y_data_sorted

acousticness             0.038250
danceability             0.655500
duration_ms         204280.000000
energy                   0.701500
instrumentalness         0.000695
key                      7.000000
liveness                 0.127500
loudness                -5.165500
speechiness              0.060100
tempo                  120.641500
valence                  0.542000
Name: top_tracks, dtype: float64

In [202]:
df_main_var = pl_rec.subtract(Y_data_sorted, axis=1)
df_main_var = df_main_var.divide(Y_data_sorted, axis=1) 

df_main_var.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence
track_id,,,,,,,,,,,
0f5IlhiVWUUq4SGHyrRxIt,24.228758,-0.784897,-0.089764,-0.942267,1389.262521,-0.571429,-0.229020,4.616688,-0.364393,-0.507483,-0.930258
5LEHlRPjGcZ5RdagAwXpHS,19.836601,-0.844394,-0.299721,-0.952530,1333.133910,-0.571429,-0.129412,5.616397,-0.207987,-0.257262,-0.890959
0kKOC50fPTnqmrYvtJInsT,24.960784,-0.797101,-0.257147,-0.992701,1223.755078,0.285714,-0.251765,5.445649,-0.367720,-0.437855,-0.908672
6PEwEkL4lzRbGMGH7j7kBB,24.803922,-0.720824,-0.115797,-0.982466,1128.768197,0.285714,-0.318431,5.692673,-0.319468,-0.401674,-0.882657
2PUoynqedBx15JbPYxzA5v,21.954248,-0.778795,0.074378,-0.983607,1229.511858,-1.000000,-0.043137,4.477108,-0.299501,-0.402196,-0.940959


In [203]:
df_main_var['variation'] = df_main_var.sum(axis=1)

df_main_var['variation'] = df_main_var['variation'].abs()

In [204]:
df_main_var

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence,variation
track_id,,,,,,,,,,,,
0f5IlhiVWUUq4SGHyrRxIt,24.228758,-0.784897,-0.089764,-0.942267,1389.262521,-0.571429,-0.229020,4.616688,-0.364393,-0.507483,-0.930258,1413.688457
5LEHlRPjGcZ5RdagAwXpHS,19.836601,-0.844394,-0.299721,-0.952530,1333.133910,-0.571429,-0.129412,5.616397,-0.207987,-0.257262,-0.890959,1354.433215
0kKOC50fPTnqmrYvtJInsT,24.960784,-0.797101,-0.257147,-0.992701,1223.755078,0.285714,-0.251765,5.445649,-0.367720,-0.437855,-0.908672,1250.434264
6PEwEkL4lzRbGMGH7j7kBB,24.803922,-0.720824,-0.115797,-0.982466,1128.768197,0.285714,-0.318431,5.692673,-0.319468,-0.401674,-0.882657,1155.809190
2PUoynqedBx15JbPYxzA5v,21.954248,-0.778795,0.074378,-0.983607,1229.511858,-1.000000,-0.043137,4.477108,-0.299501,-0.402196,-0.940959,1251.569398
...,...,...,...,...,...,...,...,...,...,...,...,...
1uKzAldJYlwje9HstlwTie,16.176471,-0.437071,-0.161088,-0.579473,-0.977836,-0.428571,-0.160784,1.473913,-0.415973,-0.235537,-0.610701,13.643350
0ZjTS5zoXFR2w9QMRweDhs,6.163399,-0.014493,0.074863,-0.111903,-1.000000,0.142857,-0.325490,0.707482,-0.439268,0.000079,0.804428,6.001954
4UmoMxbpiaXTJcKnjyUiRv,7.261438,-0.058734,0.544155,-0.103350,-0.938259,0.142857,0.270588,0.090117,-0.550749,-0.031925,-0.276753,6.349387


In [205]:
# Picking 30 least variated songs from df_main_var
rec_songs = df_main_var.nsmallest(30,'variation')

rec_songs

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence,variation
track_id,,,,,,,,,,,,
3d9DChrdc6BOeFsbrZ3Is0,0.505882,-0.147216,0.293847,-0.508197,-0.848885,-0.428571,0.105882,1.612719,-0.236273,-0.298906,-0.154982,0.104699
172Tyy70R4Cc1JWarP9DNU,-0.589542,-0.096873,0.419552,-0.399857,0.741426,-0.714286,0.466667,1.333559,-0.544093,-0.038590,-0.472325,0.105638
0LJKGUTtdzcLW9ZcUxkRXR,0.487582,-0.330282,0.705830,0.063435,-1.000000,-0.428571,-0.239216,0.579518,-0.500832,0.445630,0.047970,0.168935
3Dfy8YIxq89i84t108TvMi,0.751634,-0.099924,0.698453,0.059159,0.338451,-1.000000,-0.335686,0.370245,-0.422629,0.075832,-0.239852,0.195683
2hKdd3qO7cWr2Jo0Bcs0MA,3.000000,-0.266209,0.272435,-0.090520,-1.000000,-1.000000,0.207843,0.134837,-0.540765,-0.344637,-0.083026,0.289958
0815caqt2Lytro5EIzMufT,1.980392,-0.299771,-0.043014,-0.391304,-1.000000,0.000000,0.317647,0.497241,-0.555740,0.159651,-0.346863,0.318237
0l5TpsCL1ObiTEsHeWA0by,1.094118,-0.363844,0.306897,-0.183179,-1.000000,0.571429,-0.337255,0.419030,-0.535774,0.073917,-0.464945,0.419605
2CpMHblH2jRF9NqtaW8MKZ,1.128105,-0.626240,0.150710,0.019244,-1.000000,0.000000,1.674510,0.198722,-0.381032,-0.378091,-0.322878,0.463050
064SVQsmWl5EF0zahmzkQk,0.597386,0.035850,-0.027022,-0.211689,-0.989695,-0.571429,0.090196,1.843287,-0.474210,-0.095908,0.745387,0.942154


In [206]:
# Creates custom playlist
def custom_playlist(sp, username, playlist_name, playlist_description, isPublic):
    pl_custom = sp.user_playlist_create(username, playlist_name, public=isPublic, collaborative=False, description=playlist_description)

custom_playlist(sp, user_name, 'DJ_NumPy', 'Custom Playlist', False)

In [207]:
custom_id = sp.user_playlists(user_name)['items'][0]['id']

In [208]:
# Fills custom playlist with recommended tracks
def fill_playlist(playlist_id):
    # try:
    #     sp.user_playlist_unfollow(user=user_name, playlist_id=custom_id)
    # except:
    #     sp.playlist_add_items(playlist_id, items= rec_songs.index[0:len(rec_songs.index)])
    sp.playlist_add_items(playlist_id, items=rec_songs.index[0:len(rec_songs.index)])

In [209]:
fill_playlist(custom_id)